In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow_datasets as tfds
import tensorflow as tf

In [0]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string], '')
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

In [0]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)

train_dataset, test_dataset = dataset['train'], dataset['test']

In [0]:
encoder = info.features['text'].encoder

In [0]:
print('Vocabulary size: {}'.format(encoder.vocab_size))

Vocabulary size: 8185


In [0]:
sample_string = "Hello Tensorflow"

encoded_string = encoder.encode(sample_string)
print("Encoded string is {}".format(encoded_string))

original_string = encoder.decode(encoded_string)
print("The original string: {}".format(original_string))

Encoded string is [4025, 222, 6307, 2327, 2934]
The original string: Hello Tensorflow


In [0]:
for index in encoded_string:
  print('{}--->{}'.format(index, encoder.decode([index])))

4025--->Hell
222--->o 
6307--->Ten
2327--->sor
2934--->flow


In [0]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [0]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE, train_dataset.output_shapes)

test_dataset = test_dataset.padded_batch(BATCH_SIZE, test_dataset.output_shapes)

Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.


Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.


In [0]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(encoder.vocab_size, 64),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                             tf.keras.layers.Dense(64, activation = 'relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])

In [0]:
model.compile(loss = 'binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [0]:
history = model.fit(train_dataset, epochs = 10,
                    validation_data=test_dataset,
                    validation_steps=30)

Train on None steps, validate on 30 steps
Epoch 1/10
391/391 [==============================] - 1592s 4s/step - loss: 0.6626 - acc: 0.5808 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/10
391/391 [==============================] - 1591s 4s/step - loss: 0.3447 - acc: 0.8580 - val_loss: 0.3405 - val_acc: 0.8568
Epoch 3/10
 97/391 [======>.......................] - ETA: 19:50 - loss: 0.2385 - acc: 0.9112

In [0]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))